In [1]:
import numpy as np
import torch

In [2]:
from ddnn.nn import Initializer

Initializer("random_uniform")((4, 5))
Initializer("random_normal")((4, 5))
Initializer("glorot_uniform")((4, 5))
Initializer("glorot_normal")((4, 5))
Initializer("he_uniform", fan_mode="fan_in")((4, 5))
Initializer("he_normal", fan_mode="fan_out")((4, 5));

In [3]:
from ddnn.nn import ActivationFunction


def assert_activation_function(fname, pytorch_equivalent):
    inputs = np.array([0, -1, 1, -1e5, 1e5, 1e-5, -1e-5])
    tinput = torch.Tensor(inputs).requires_grad_()
    afun = ActivationFunction(fname)

    outputs = afun.foward(inputs)
    expected = pytorch_equivalent(tinput)
    assert np.allclose(outputs, expected.detach()), f"{fname} func"

    expected.sum().backward()
    assert np.allclose(
        afun.backward(np.ones_like(outputs)), tinput.grad
    ), f"{fname} grad"


assert_activation_function("ReLU", torch.nn.ReLU())
assert_activation_function("logistic", torch.nn.Sigmoid())
assert_activation_function("tanh", torch.nn.Tanh())
# overflow error in logistic function can be safely ignored

/home/david/uni/ml/Machine-Learning-Project/ddnn/nn/activation.py:86: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


In [4]:
from ddnn.nn import LossFunction

inputs = np.array([0, 1, 0, 1])
labels = np.array([0, 0, 1, 1])
tinput = torch.Tensor(inputs).requires_grad_()
tlabels = torch.Tensor(labels)

tinput.grad = None
lfun = LossFunction("MSE")
outputs = lfun.foward(inputs.reshape(inputs.size, 1), labels)
expected = torch.nn.MSELoss()(tinput, tlabels)
assert np.allclose(outputs, expected.detach()), "MSE func"
expected.backward()
assert np.allclose(lfun.backward().squeeze(), tinput.grad), "MSE grad"

# TODO add manual check for accuracy and MEE

In [5]:
from ddnn.nn import LinearLayer

tlayer = torch.nn.Linear(10, 2)
layer = LinearLayer((10, 2))
layer.params.weights[:] = tlayer.weight.detach()
layer.params.bias[:] = tlayer.bias.detach()

tinputs = torch.rand(8, 10, requires_grad=True)
tlabels = torch.rand(8, 2)
inputs = tinputs.detach().numpy()
labels = tlabels.detach().numpy()

tpreds = tlayer(tinputs)
preds = layer.foward(inputs)
assert np.allclose(preds, tpreds.detach()), "Linear Forward"

tpreds.sum().backward()
tigrad = tinputs.grad
igrad = layer.backward(np.ones_like(preds))
assert np.allclose(igrad, tigrad.detach()), "Linear Backward [Input]"
assert np.allclose(layer.grads.weights, tlayer.weight.grad), "Linear Backward [Weights]"
assert np.allclose(layer.grads.bias, tlayer.bias.grad), "Linear Backward [Bias]"

In [6]:
from ddnn.nn import Optimizer
from ddnn.utils import Parameter

params = Parameter(np.array([10, 10]), np.array([10]))
grads = Parameter(np.array([1, -1]), np.array([-1]))
state = Parameter(np.array([0.5, -1.5]), np.array([-0.5]))

expected = Parameter(
    0.5 * state.weights - 0.2 * params.weights - 1 * grads.weights,
    0.5 * state.bias - 1 * grads.bias,
)

update, state = Optimizer(
    "SGD", learning_rate=1, l2_coefficient=0.1, momentum_coefficient=0.5
)(1)(params, grads, state)
assert np.allclose(update.weights, state.weights) and np.allclose(
    update.weights, expected.weights
), "SGD [Weight]"
assert np.allclose(update.bias, state.bias) and np.allclose(
    update.bias, expected.bias
), "SGD [Bias]"